In [1]:
import pandas as pd
from ortools.sat.python import cp_model
from datetime import datetime, timedelta

In [2]:
import sys
sys.path.append('/home/cayena/Projeto_thefamily/TeacherScheduler/')

In [3]:
from streamlit_app.utils import replicate_row,base_selection,expand_rows,clean_data
from streamlit_app.teacher_alocation import TeacherScheduler

In [4]:
professores_raw = pd.read_excel('/home/cayena/Downloads/professores_final.xlsx')
professores_raw.columns = professores_raw.columns.astype(str)

aulas_raw = pd.read_excel('/home/cayena/Downloads/MODELO TESTE CARLOS.xlsx')

In [5]:
aulas_simples,doub,tri=base_selection(aulas_raw)

# transformando aulas duplas/triplas em 2/3 linhas
aulas_duplicadas = expand_rows(doub, lambda row: replicate_row(row, times=2))
aulas_triplicadas = expand_rows(tri, lambda row: replicate_row(row, times=3))

# juntando todas as linhas
df_tratado = pd.concat([aulas_simples, aulas_duplicadas, aulas_triplicadas], ignore_index=True)

df_final = clean_data(df_tratado)

df_final.head()

,nome grupo,horario,unidade,dias da semana,stage,modalidade,grupo,n aulas,teacher,status,restricoes_professor,ultimo_professor,penultimo_professor,STAGE,PARAG FINAL,nº 1 sempre,intenviso,horario_tratado
0,ABU DHABI EAD - 4X,19:00:00,Vicentina,SEGUNDA,ESTAGIO_11,Inglês,Grupo,4.0,-,ONLINE,NaN,NaN,NaN,11,1246,1,0,1900-01-01 19:00:00
2,ABU DHABI EAD - 4X,19:00:00,Vicentina,TERÇA,ESTAGIO_11,Inglês,Grupo,4.0,-,ONLINE,NaN,NaN,NaN,11,1246,1,0,1900-01-01 19:00:00
4,ABU DHABI EAD - 4X,19:00:00,Vicentina,QUARTA,ESTAGIO_11,Inglês,Grupo,4.0,-,ONLINE,NaN,NaN,NaN,11,1246,1,0,1900-01-01 19:00:00
6,ABU DHABI EAD - 4X,19:00:00,Vicentina,QUINTA,ESTAGIO_11,Inglês,Grupo,4.0,-,ONLINE,NaN,NaN,NaN,11,1246,1,0,1900-01-01 19:00:00
8,ANDERSON VIP EAD - 1X,15:30:00,Vicentina,SEXTA,ESTAGIO_2,Inglês,VIP,1.0,BRUNO MORGILLO,ONLINE,NaN,NaN,NaN,2,170,1,0,1900-01-01 15:30:00


In [6]:
Ts = TeacherScheduler(df_final, professores_raw)
base_alocada = Ts.schedule_teachers()
base_alocada

,Professor,nome grupo
0,ALINE,ABU DHABI EAD - 4X
1,MARIA,ANDERSON VIP EAD - 1X
2,MARIA,ANDRE VIP CONV EAD - 2X
3,BARBARA,AUCKLAND P - 4X
4,ALINE,BAKERSFIELD P - 4X
...,...,...
131,FERNANDO,GIBRALTAR P - 3X
132,MARIA,MONTERREY EAD - 3X
133,LEONARDO,NEBRASKA P - 3X
134,CLAUDIO,SCOTT EAD - 3X


In [7]:
final = pd.merge(aulas_raw, base_alocada, on='nome grupo', how='left')

In [8]:
final[final['Professor'].isnull()]

,nome grupo,horario,unidade,dias da semana,stage,modalidade,grupo,n aulas,teacher,status,restricoes_professor,ultimo_professor,penultimo_professor,STAGE,PARAG FINAL,nº 1 sempre,Professor
98,NORTHAMPTOM P - 3X,08:00:00,Satélite,Saturday - Triple,4,Inglês,Grupo,3.0,-,PRESENCIAL,NaN,NaN,NaN,4,390,1,NaN
115,SAN JUAN EAD - 3X,08:00:00,Jardim,Saturday - Triple,4,Espanhol,Grupo,3.0,-,ONLINE,NaN,NaN,NaN,4,421,1,NaN
